In [19]:
import os
import pandas as pd
import json

cwd = os.getcwd()
tea = pd.read_excel(cwd + "/tea_trade_2022.xlsx", sheet_name="Trades")


In [20]:
naRate = pd.DataFrame({
    "Year" : tea.Year.unique(),
    "NA_Rate" : float(0) * len(tea.Year.unique())
})
naRate
for year in tea.Year.unique():
    indice = tea[tea["Year"] == year]["Value (1000USD)"].isna().value_counts()
    na = indice[1] / sum(indice) * 100
    naRate.loc[naRate["Year"] == year, "NA_Rate"] = na

naRate

/var/folders/m5/2zm1zfkj0f34zf9y6m_q_6yw0000gn/T/ipykernel_7679/404559807.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  na = indice[1] / sum(indice) * 100


,Year,NA_Rate
0,2018,1.340397
1,2019,1.273567
2,2020,1.430182
3,2021,1.756898
4,2022,1.607919


In [21]:
selectedYear = 2022
tea20 = tea[tea["Year"] == selectedYear][["Exporter", "Importer", "Value (1000USD)"]]\
    .rename(columns={"Value (1000USD)": "Value"})\
    .fillna(0)\
    .reset_index(drop=True)

In [22]:
## Create a top list of exporters and importers
topImEx = pd.DataFrame({"Exporter": tea20.groupby("Exporter").agg({"Value": "sum"}).sort_values(by="Value", ascending=False).head(5).index, 
                        "Importer": tea20.groupby("Importer").agg({"Value": "sum"}).sort_values(by="Value", ascending=False).head(5).index})
topImEx20 = tea20.copy()
def replaceCountries(country, selectedList):
    return country if country in list(selectedList) else "other"

## Filter out the rest of the list to be others
topImEx20['Exporter'] = topImEx20['Exporter'].apply(replaceCountries, selectedList = topImEx["Exporter"])
topImEx20['Importer'] = topImEx20['Importer'].apply(replaceCountries, selectedList = topImEx["Importer"])
topImEx20['Exporter'].replace('other', 'Other Exporter', inplace = True)
topImEx20['Importer'].replace('other', 'Other Importer', inplace = True)
topImEx20 = topImEx20.groupby(['Exporter', 'Importer'], as_index=False)['Value'].sum()

In [23]:
topImEx20

,Exporter,Importer,Value
0,China,Other Importer,1.405580e+06
1,China,Pakistan,8.409374e+03
2,China,Russian Federation,5.173228e+04
3,China,United Arab Emirates,9.152616e+03
4,China,United Kingdom,1.447736e+04
5,China,United States,7.064613e+04
6,India,Other Importer,4.234638e+05
7,India,Pakistan,4.004814e+02
8,India,Russian Federation,9.110677e+04
9,India,United Arab Emirates,1.224779e+05


In [24]:
topEx = topImEx20.groupby(by="Exporter").sum().sort_values(by="Value",ascending=False).reset_index().iloc[1:6,:]['Exporter']
topIm = topImEx20.groupby(by="Importer").sum().sort_values(by="Value",ascending=False).reset_index().iloc[1:6,:]['Importer']
topEx

1                   China
2                   Kenya
3               Sri Lanka
4                   India
5    United Arab Emirates
Name: Exporter, dtype: object

In [25]:
nodes = []
nodes.extend(topEx.tolist())
nodes.append("Other Exporter")
nodes.extend(topIm.tolist())
nodes.append("Other Importer")
nodes

['China',
 'Kenya',
 'Sri Lanka',
 'India',
 'United Arab Emirates',
 'Other Exporter',
 'Pakistan',
 'United States',
 'Russian Federation',
 'United Arab Emirates',
 'United Kingdom',
 'Other Importer']

In [26]:
sankey_data = {"nodes":[], "links":[]}

nodes = pd.DataFrame(
    nodes,
    columns=["country"]
)

for index, row in nodes.iterrows():
    sankey_data['nodes'].append({"node":index, "name": row['country']})

for index, row in topImEx20.iterrows():
    exporter_index = int(nodes[nodes["country"] == row["Exporter"]].index[0])
    importer_index = int(nodes[nodes["country"] == row["Importer"]].index[0])
    value = int(row["Value"])

    sankey_data["links"].append({
        "source": exporter_index,
        "target": importer_index,
        "value": value
    })

In [27]:
with open('tea_trade_2022.json', 'w') as json_file:
    json.dump(sankey_data, json_file, indent=2)